# How to get MRN News Analytics via WebSocket API with Python - Deployed RTDS Connection

**Last Update**: Nov 2025

## Prerequisite

This article/notebook supports all [Machine Readable News Analytics (TRNA)](https://www.lseg.com/en/data-analytics/financial-news-service/machine-readable-news) data data consumption from The Real-Time with the WebSocket API. However, the data model description is focusing on the News Analytics (TRNA) data processing only. 

I highly recommend you check the  [WebSocket API Tutorials](https://developers.lseg.com/en/api-catalog/real-time-opnsrc/websocket-api/tutorials) page if you are not familiar with WebSocket API. The Tutorials page provides a step-by-step guide (connect, log in, request data, parse data, etc) for developers who are interested in developing a WebSocket application to consume real-time data from  Real-Time. 

If you are focusing on the Real-Time News, please check the following GitHub repositories
- [LSEG-API-Samples/Example.WebSocketAPI.Python.MRN](https://github.com/LSEG-API-Samples/Example.WebSocketAPI.Python.MRN).
- [LSEG-API-Samples/Example.WebSocketAPI.Python.MRN.RTO](https://github.com/LSEG-API-Samples/Example.WebSocketAPI.Python.MRN.RTO)

The notebook file provides a step-by-step guide (connect, log in, request data, parse data, etc) for developers who are interested in developing a WebSocket application to consume real-time data from the deployed Refinitiv Real-Time Distribution System (RTDS). 

If you are using the Real-Time - Optimized (RTO), please check the [mrn_trna_notebook_app_rto.ipynb](./mrn_trna_notebook_app_rto.ipynb) notebook file.

## News Analytics and Machine Readable News Overview

[Machine Readable News Analytics (TRNA)](https://www.lseg.com/en/data-analytics/financial-news-service/machine-readable-news) provides real-time numerical insight into the events on multiple news sources, in a format that can be directly consumed by algorithmic trading systems. TRNA enables algorithms to exploit the power of news to seize opportunities, capitalize on market inefficiencies, and manage event risk.

TRNA is published via the Real-Time Platform as part of Machine Readable News (MRN) data model. MRN is an advanced service for automating the consumption and systematic analysis of news. It delivers deep historical news archives, ultra-low latency structured news, and news analytics directly to your applications. 

### MRN Data behavior

RN is published over Refinitiv Real-Time using an Open Message Model (OMM) envelope in News Text Analytics domain messages. The Real-time News content set is made available over MRN_STORY RIC. The content data is contained in a FRAGMENT field that has been compressed and potentially fragmented across multiple messages, to reduce bandwidth and message size.

A FRAGMENT field has a different data type based on a connection type:

- RSSL connection (RTSDK [C++](https://developers.lseg.com/en/api-catalog/real-time-opnsrc/rt-sdk-cc)/[C#](https://developers.lseg.com/en/api-catalog/real-time-opnsrc/rt-sdk-csharp)/[Java](https://developers.lseg.com/en/api-catalog/real-time-opnsrc/rt-sdk-java)): BUFFER type
- WebSocket connection: Base64 ASCII string

The data goes through the following series of transformations:

1. The core content data is a UTF-8 JSON string
2. This JSON string is compressed using gzip
3. The compressed JSON is split into several fragments (BUFFER or Base64 ASCII string) which each fit into a single update message
4. The data fragments are added to an update message as the FRAGMENT field value in a FieldList envelope


<img src="images/trna_process.png"/>

Therefore, to parse the core content data, the application will need to reverse this process. The WebSocket application also needs to convert a received Base64 string in a FRAGMENT field to bytes data before further process this field. This application uses Python [base64](https://docs.python.org/3/library/base64.html) and [zlib](https://docs.python.org/3/library/zlib.html) modules to decode Base64 string and decompress JSON string. 

### MRN Data model

Five fields, as well as the RIC itself, are necessary to determine whether the entire item has been received in its various fragments and how to concatenate the fragments to reconstruct the item:
* MRN_SRC: identifier of the scoring/processing system that published the FRAGMENT
* GUID: a globally unique identifier for the data item. All messages for this data item will have the same GUID values.
* FRAGMENT: compressed data item fragment, itself
* TOT_SIZE: total size in bytes of the fragmented data
* FRAG_NUM: sequence number of fragments within a data item. This is set to 1 for the first fragment of each item published and is incremented for each subsequent fragment for the same item.

A single MRN data item publication is uniquely identified by the combination of RIC, MRN_SRC, and GUID.

#### Fragmentation
For a given RIC-MRN_SRC-GUID combination, when a data item requires only a single message, then TOT_SIZE will equal the number of bytes in the FRAGMENT, and FRAG_NUM will be 1.

When multiple messages are required, then the data item can be deemed as fully received once the sum of the number of bytes of each FRAGMENT equals TOT_SUM. The consumer will also observe that all FRAG_NUM range from 1 to the number of the fragment, with no intermediate integers, skipped. In other words, a data item transmitted over three messages will contain FRAG_NUM values of 1, 2, and 3.

#### Compression
The FRAGMENT field is compressed with gzip compression, thus requiring the consumer to decompress to reveal the JSON plain-text data in that FID.

When an MRN data item is sent in multiple messages, all the messages must be received and their FRAGMENTs concatenated before being decompressed. In other words, the FRAGMENTs should not be decompressed independently of each other.

The decompressed output is encoded in UTF-8 and formatted as JSON.

If you are not familiar with MRN concept, please visit the following resources which will give you a full explanation of the MRN data model and implementation logic:

* [Webinar Recording: Introduction to Machine Readable News](https://developers.lseg.com/news#news-accordion-nid-12045)
* [Introduction to Machine Readable News (MRN) with Enterprise Message API (EMA)](https://developers.lseg.com/en/article-catalog/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
* [MRN Data Models and Refinitiv Real-Time SDK Implementation Guide](https://developers.lseg.com/en/api-catalog/real-time-opnsrc/rt-sdk-java/documentation#mrn-data-models-and-elektron-implementation-guide).
* [Introduction to Machine Readable News with WebSocket API](https://developers.lseg.com/en/article-catalog/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).
* [How to get MRN News Analytics Data via WebSocket API](https://developers.lseg.com/en/article-catalog/article/how-to-get-mrn-news-analytics-data-via-elektron-websocket-api).

In [3]:
# #uncomment if you do not have requests and websocket-client (version 0.49 and above) installed\n
# #Install requests and websocket-client packages in a current Jupyter kernal\n

import sys

# !{sys.executable} -m pip install requests
# !{sys.executable} -m pip install websocket-client

In [4]:

import time
import getopt
import socket
import json
import websocket
import threading
from threading import Thread, Event
import base64
import zlib

In [5]:
# Refinitiv Real-Time Advanced Distribution Server connection variables

hostname = '172.31.94.131'
port = '15000'
user = 'user'
app_id = '256'
position = socket.gethostbyname(socket.gethostname())
login_id = 1

In [6]:
# WebSocket connections Variables

web_socket_app = None
web_socket_open = False
_news_envelopes = []

# keeps decompress news JSON messaage
_trna_messages = []

### MRN Process Code

The MRN data can be subscribed with the *NewsTextAnalytics* domain and MRN-specific RIC name as following:
- *MRN_TRNA*: News Analytics: Company and C&E assets
- *MRN_TRNA_DOC*: News Analytics: Macroeconomic News & events
- *MRN_STORY*: Real-time News
- *MRN_TRSI*: News Sentiment Indices

In [7]:
# MRN variables

mrn_domain = 'NewsTextAnalytics'
mrn_item = 'MRN_TRNA'

def send_mrn_request(ws):
    """ Create and send MRN request """
    mrn_req_json = {
        'ID': 2,
        "Domain": mrn_domain,
        'Key': {
            'Name': mrn_item
        }
    }

    ws.send(json.dumps(mrn_req_json))
    print("SENT:")
    print(json.dumps(mrn_req_json, sort_keys=True, indent=2, separators=(',', ':')))

### Initial Refresh Message
The Initial Refresh response does not contain any NTA data, all the fields related to news item and fragment are empty or 0. It contains only the relevant feed related or other static Fields. 

The application can just print out each incoming field data in a console for informational purpose or just ignore it.

In [8]:
# Process FieldList, Refresh and Status messages.

def decodeFieldList(fieldList_dict):
    for key, value in fieldList_dict.items():
        print("Name = %s: Value = %s" % (key, value))

def processRefresh(ws, message_json):

    print("RECEIVED: Refresh Message")
    decodeFieldList(message_json["Fields"])

def processStatus(ws, message_json):  # process incoming status message
    print("RECEIVED: Status Message")
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

### MRN News Update messages Process Code

The updates contain only fields related to the item and the fragment. They do not contain any of the static or per-feed fields. The updates are not cached or conflated.

#### First Update
The first update contains all the fields related to the item and the first fragment, subsequent updates only contain the fields relating to the fragment they contain. The FRAG_NUM FID is set to 1 for the first Update of each item and is incremented in each subsequent Update for that item. This allows you to you to detect a missing fragment (and ensure correct order of the fragments for re-assembly). 


#### Subsequent Update and Multi Fragment Items
The subsequent update contains the fields necessary to identify the MRN data item, the order of this fragment among all the fragments for this item, and the fragment itself. The other point to note is that (for a Multi fragment item), Update messages with FRAG_NUM >1 will have fewer FIDs as the metadata is included in the first Update message (FRAG_NUM=1) for that item

#### News Fragments simple handle logic


<img src="images/mrn_flow_reconstruct.png"/>


In [9]:
def processMRNUpdate(ws, message_json):  # process incoming News Update messages

    fields_data = message_json["Fields"]
    # Dump the FieldList first (for informational purposes)
    # decodeFieldList(message_json["Fields"])

    # declare variables
    tot_size = 0
    guid = None

    try:
        # Get data for all requried fields
        fragment = base64.b64decode(fields_data["FRAGMENT"])
        frag_num = int(fields_data["FRAG_NUM"])
        guid = fields_data["GUID"]
        mrn_src = fields_data["MRN_SRC"]

        #print("GUID  = %s" % guid)
        #print("FRAG_NUM = %d" % frag_num)
        #print("MRN_SRC = %s" % mrn_src)

        if frag_num > 1:  # We are now processing more than one part of an envelope - retrieve the current details
            guid_index = next((index for (index, d) in enumerate(
                _news_envelopes) if d["guid"] == guid), None)
            envelop = _news_envelopes[guid_index]
            if envelop and envelop["data"]["mrn_src"] == mrn_src and frag_num == envelop["data"]["frag_num"] + 1:
                print("process multiple fragments for guid %s" %
                      envelop["guid"])

                #print("fragment before merge = %d" % len(envelop["data"]["fragment"]))

                # Merge incoming data to existing news envelop and getting FRAGMENT and TOT_SIZE data to local variables
                fragment = envelop["data"]["fragment"] = envelop["data"]["fragment"] + fragment
                envelop["data"]["frag_num"] = frag_num
                tot_size = envelop["data"]["tot_size"]
                print("TOT_SIZE = %d" % tot_size)
                print("Current FRAGMENT length = %d" % len(fragment))

                # The multiple fragments news are not completed, waiting.
                if tot_size != len(fragment):
                    return None
                # The multiple fragments news are completed, delete assoiclate GUID envelop
                elif tot_size == len(fragment):
                    del _news_envelopes[guid_index]
            else:
                print("Error: Cannot find fragment for GUID %s with matching FRAG_NUM or MRN_SRC %s" % (
                    guid, mrn_src))
                return None
        else:  # FRAG_NUM = 1 The first fragment
            tot_size = int(fields_data["TOT_SIZE"])
            print("FRAGMENT length = %d" % len(fragment))
            # The fragment news is not completed, waiting and add this news data to envelop object.
            if tot_size != len(fragment):
                print("Add new fragments to news envelop for guid %s" % guid)
                _news_envelopes.append({  # the envelop object is a Python dictionary with GUID as a key and other fields are data
                    "guid": guid,
                    "data": {
                        "fragment": fragment,
                        "mrn_src": mrn_src,
                        "frag_num": frag_num,
                        "tot_size": tot_size
                    }
                })
                return None

        # News Fragment(s) completed, decompress and print data as JSON to console
        if tot_size == len(fragment):
            print("decompress News FRAGMENT(s) for GUID  %s" % guid)
            decompressed_data = zlib.decompress(fragment, zlib.MAX_WBITS | 32)
            
            json_news = json.loads(decompressed_data)
            _trna_messages.append(json_news)
            print("News = %s" % json_news)

    except KeyError as keyerror:
        print('KeyError exception: ', keyerror)
    except IndexError as indexerror:
        print('IndexError exception: ', indexerror)
    except binascii.Error as b64error:
        print('base64 decoding exception:', b64error)
    except zlib.error as error:
        print('zlib decompressing exception: ', error)
    # Some console environments like Windows may encounter this unicode display as a limitation of OS
    except UnicodeEncodeError as encodeerror:
        print("UnicodeEncodeError exception. Cannot decode unicode character for %s in this enviroment: " %
              guid, encodeerror)
    except Exception as e:
        print('exception: ', sys.exc_info()[0])

### JSON-OMM Process functions

In [10]:
def process_message(ws, message_json):
    """ Parse at high level and output JSON of message """
    message_type = message_json['Type']

    if message_type == "Refresh":
        if "Domain" in message_json:
            message_domain = message_json["Domain"]
            if message_domain == "Login":
                process_login_response(ws, message_json)
            elif message_domain:
                processRefresh(ws, message_json)
    elif message_type == "Update":
        if "Domain" in message_json and message_json["Domain"] == mrn_domain:
            processMRNUpdate(ws, message_json)
    elif message_type == "Status":
        processStatus(ws, message_json)
    elif message_type == "Ping":
        pong_json = {'Type': 'Pong'}
        ws.send(json.dumps(pong_json))
        print("SENT:")
        print(json.dumps(pong_json, sort_keys=True,
                         indent=2, separators=(',', ':')))


def process_login_response(ws, message_json):
    """ Send item request """
    send_mrn_request(ws)


def send_login_request(ws):
    """ Generate a login request from command line data (or defaults) and send """
    login_json = {
        'ID': 1,
        "Domain": 'Login',
        'Key': {
            'Name': '',
            'Elements': {
                'ApplicationId': '',
                'Position': ''
            }
        }
    }

    login_json['Key']['Name'] = user
    login_json['Key']['Elements']['ApplicationId'] = app_id
    login_json['Key']['Elements']['Position'] = position

    ws.send(json.dumps(login_json))
    print("SENT:")
    print(json.dumps(login_json, sort_keys=True, indent=2, separators=(',', ':')))

### WebSocket Process functions

In [11]:
def on_message(ws, message):
    """ Called when message received, parse message into JSON for processing """
    print("RECEIVED: ")
    message_json = json.loads(message)
    print(json.dumps(message_json, sort_keys=True, indent=2, separators=(',', ':')))

    for singleMsg in message_json:
        process_message(ws, singleMsg)
        
def on_error(ws, error):
    """ Called when websocket error has occurred """
    print(error)
    
def on_close(ws,close_status_code, close_msg):
    """ Called when websocket is closed """
    global web_socket_open
    print("WebSocket Closed")
    web_socket_open = False
    
def on_open(ws):
    """ Called when handshake is complete and websocket is open, send login """

    print("WebSocket successfully connected!")
    global web_socket_open
    web_socket_open = True
    send_login_request(ws)

if __name__ == "__main__":
    # Start websocket handshake
    ws_address = "ws://{}:{}/WebSocket".format(hostname, port)
    print("Connecting to WebSocket " + ws_address + " ...")
    web_socket_app = websocket.WebSocketApp(ws_address, header=['User-Agent: Python'],
                                            on_message=on_message,
                                            on_error=on_error,
                                            on_close=on_close,
                                            subprotocols=['tr_json2'])
    web_socket_app.on_open = on_open

    # Event loop
    #wst = threading.Thread(target=web_socket_app.run_forever)
    wst = threading.Thread(target=web_socket_app.run_forever, kwargs={'sslopt': {'check_hostname': False}})
    wst.start()

    time.sleep(90)
    web_socket_app.close()

Connecting to WebSocket ws://172.31.94.131:15000/WebSocket ...
WebSocket successfully connected!
SENT:
{
  "Domain":"Login",
  "ID":1,
  "Key":{
    "Elements":{
      "ApplicationId":"256",
      "Position":"192.168.68.118"
    },
    "Name":"user"
  }
}
RECEIVED: 
[
  {
    "Domain":"Login",
    "Elements":{
      "MaxMsgSize":65535,
      "PingTimeout":30
    },
    "ID":1,
    "Key":{
      "Elements":{
        "AllowSuspectData":1,
        "ApplicationId":"256",
        "ApplicationName":"ADS",
        "Position":"192.168.68.118",
        "ProvidePermissionExpressions":1,
        "ProvidePermissionProfile":0,
        "SingleOpen":1,
        "SupportBatchRequests":7,
        "SupportEnhancedSymbolList":1,
        "SupportOMMPost":1,
        "SupportOptimizedPauseResume":0,
        "SupportPauseResume":0,
        "SupportStandby":1,
        "SupportStandbyMode":3,
        "SupportViewRequests":1
      },
      "Name":"user"
    },
    "State":{
      "Data":"Ok",
      "Stream":"Ope

In [12]:
_trna_messages

[{'analytics': {'analyticsScores': [{'assetClass': 'CMPNY',
     'assetCodes': ['P:4295914598', 'R:BKNG.O', 'R:BKNG.OQ'],
     'assetId': '4295914598',
     'assetName': 'Booking Holdings Inc',
     'brokerAction': 'UNDEFINED',
     'firstMentionSentence': 0,
     'linkedIds': [],
     'noveltyCounts': [{'itemCount': 0, 'window': '12H'},
      {'itemCount': 0, 'window': '24H'},
      {'itemCount': 0, 'window': '3D'},
      {'itemCount': 0, 'window': '5D'},
      {'itemCount': 0, 'window': '7D'}],
     'priceTargetIndicator': 'UNDEFINED',
     'relevance': 0.0936586,
     'sentimentClass': 1,
     'sentimentNegative': 0.0398969,
     'sentimentNeutral': 0.126483,
     'sentimentPositive': 0.83362,
     'sentimentWordCount': 471,
     'volumeCounts': [{'itemCount': 2, 'window': '12H'},
      {'itemCount': 5, 'window': '24H'},
      {'itemCount': 7, 'window': '3D'},
      {'itemCount': 19, 'window': '5D'},
      {'itemCount': 34, 'window': '7D'}]}],
   'newsItem': {'bodySize': 3264,
    '

This Notebook application connects to Refinitiv Real-Time Advanced Distribution Server via the WebSocket connection, then consumes TRNA Data as MRN data domain. When the Notebook receives TRNA data from Real-Time Advanced Distribution Server, it assembles, decodes MRN textual News Analytics message and keeps them in ```_trna_messages``` [list](https://docs.python.org/3.7/tutorial/datastructures.html#more-on-lists) variable. You can get each TRNA JSON message and associate analytics fields from this variable.

## News Analytics Data Model Overview

The structure of the data within each data feed is defined in the following sections. After assembly and decompression, the data appears as JSON in UTF-8.

You can find the full detail of News Analytics Data Models and Fields in *User Guide* section of [My Refinitiv's News Analytics page](https://my.refinitiv.com/content/mytr/en/product/thomson-reuters-news-analytics.html).

The News Analytics feed has three top-level items:
- *id*: The value of this field is in ```[feedFamilyCode]:[sourceId]``` format.
- *analytics*: Analytics Groups sub-group containing the analytics scores
- *newsItem*: This group contains metadata sourced directly from the STORY item, in contrast to the newsItem group also inside the analytics group that contains data derived from the TRNA scoring.

In [13]:
analytics_id  = _trna_messages[0]["id"]
print("Analytics id: ", analytics_id)

Analytics id:  tr:PNAMPZ1ta_2511052mv01409rpi9NMNF7W9dgMEThL3cLnJRFwFW3S


### News Item Group (Top-Level Group)

The News Analytics feed contains two news item groups. This top-level group contains values which are contained within the news item being processed; the other group (above section) within the analytics group contains values derived from the news item by the analytics system.

Because the fields below are sourced from the incoming news item data and mapped to the below fields, those mappings can vary by the feedFamilyCode value. Those mappings are distinguished in the Notes section in the below table.

Example Fields:
- ```dataType```: The broad type of data the news item belongs to. One of "News", "Social"
- ```feedFamilyCode```: A code that identifies the family of feeds the news item came from. Thomson Reuters feeds = "tr"
- ```headline```: The headline text of the news item.
- ```sourceTimestamp```: UTC timestamp of this news item. Millisecond precision. The source of this data varies by the feedFamilyCode value.
- ```provider```: Identifier for the organization which provided the news item. The source of this data varies by the feedFamilyCode value.
    * "tr": from provider field
    * "mrvr": from sourceName or publisher field
- ```urgency```: Differentiates story types. 1: alert, 3: article
    

In [14]:
news_item  = _trna_messages[0]["newsItem"]

In [15]:
print("dataType: ", news_item["dataType"])
print("Headline: ", news_item["headline"])
print("sourceTimestamp: ", news_item["sourceTimestamp"])
print("feedFamilyCode: ", news_item["feedFamilyCode"])
print("provider: ", news_item["provider"][3:]) # news_item["provider"] == NS:RTRS
print("urgency: ", news_item["urgency"], " : ", 
      (lambda item_type: "alert" if 1 else "article")(news_item["urgency"]))

dataType:  News
Headline:  Agoda Collaborates with the Philippines Department of Tourism to Enhance Hospitality through Technology
sourceTimestamp:  2025-11-05T06:20:12.850Z
feedFamilyCode:  tr
provider:  PRN
urgency:  3  :  alert


In [16]:
analytic_scores_group = _trna_messages[0]["analytics"]["analyticsScores"]

### Analytics Score Group

Each analytics score group contains all the analytics information derived from the news item for a specific asset as a simple group of named values.

Example Fields:
- ```assetClass```: The broad class that the asset belongs to. Also describes the type of TRTS sentiment engine used in the scoring.
    * Either "CMPNY" for a company or "COM" for a commodity.
    * Set to “CMPNY” for document-level scores because of use of the same scoring engine as used for company-level scores.
- ```assetCodes```: List of prefixed codes, in conjunction with assetId field below, which identify the asset within various symbologies.

    * By assetClass value:
    “CMPNY”: “P:” prefix for PermID and “R:” for RIC. Can contain multiple RICs for a single company, including the primary one and those tagged to the news item.
    
    * “COM”: “N2” for topic code
- ```assetId```: Primary identifier for the asset. PermID for company and topic code for commodity.
- ```assetName```: A human readable name for the asset, used as an identifier for unknown entity scoring.
- ```brokerAction```: Denotes whether the news item is reporting the action of a broker recommendation for a security issued by the company.

    * One of "UPGRADE", "DOWNGRADE", "MAINTAIN", "BROKER", "INITIATE", "UNDEFINED"
- ```firstMentionSentence```: The first sentence, starting with the headline, in which the scored asset is mentioned. Thus, a value of 1 denotes the headline, 2 the first sentence of the story body, 3 the second sentence, etc.
- ```priceTargetIndicator```: When the news item is a price target indicator for the asset.

    * One of "INCREASE", "DECREASE", "MAINTAIN", "BROKER", "INITIATE", "UNDEFINED"
    * Set to “UNDEFINED” for all Japanese-language and document-level scores.
- ```relevance```: A decimal number indicating the relevance of the news item to the asset. It ranges from 0 to 1.
- ```sentimentClass```: This field indicates the predominant sentiment class for this news item with respect to this asset. The indicated class is the one with the highest probability.
    * 1: Positive
    * 0: Neutral 
    * -1: Negative
- ```sentimentNegative```: The probability that the sentiment of the news item was negative for the asset.
- ```sentimentNeutral```: The probability that the sentiment of the news item was neutral for the asset.
- ```sentimentPositive```: The probability that the sentiment of the news item was positive for the asset.
- ```sentimentWordCount```: The number of lexical tokens (words and punctuation) in the sections of the item text that are deemed relevant to the asset.

#### TRNA Analytics Group Processing functions

In [17]:
def get_permid(asset_codes):
    for code in asset_codes:
        if code[:2] == "P:":
            return code[2:]

def get_company(asset_codes):
    company = [code[2:] for code in asset_codes if code[:2] == "R:"]
    return " ".join(company)

def get_topic_code(asset_codes):
    topic = [code[3:] for code in asset_codes if code[:3] == "N2:"]
    return " ".join(topic)

In [18]:
# Analytics Group Fields

asset_class = None
asset_codes = None
sentiment_class = {-1: 'Negative', 0: 'Neutral', 1: 'Postivie'}

for analytic_score in analytic_scores_group:
    if analytic_score["assetClass"]:
        asset_class = analytic_score["assetClass"]
        asset_codes = analytic_score["assetCodes"]
        print("assetClass: ", asset_class)
        print("assetCodes: ", asset_codes)
        if asset_class == "CMPNY":
            print("PermID: ", get_permid(asset_codes))
            print("Co: ", get_company(asset_codes))
        elif asset_class == "COM":
            print("Topic Codes: ", get_topic_code(asset_codes))
        print("assetId: ", analytic_score["assetId"])
        print("assetName: ", analytic_score["assetName"])
        print("brokerAction: ", analytic_score["brokerAction"])
        print("relevance: ",analytic_score["relevance"])
        print("sentimentClass: ", analytic_score["sentimentClass"], 
              ":", sentiment_class[analytic_score["sentimentClass"]] )
        print("sentimentPositive: ", analytic_score["sentimentPositive"])
        print("sentimentNeutral: ", analytic_score["sentimentNeutral"])
        print("sentimentNegative: ", analytic_score["sentimentNegative"])
        print("priceTargetIndicator: ", analytic_score["priceTargetIndicator"])
        print("firstMentionSentence: ", analytic_score["firstMentionSentence"])
        print("sentimentWordCount: ", analytic_score["sentimentWordCount"])
        print("--------------------------------------------------------")

assetClass:  CMPNY
assetCodes:  ['P:4295914598', 'R:BKNG.O', 'R:BKNG.OQ']
PermID:  4295914598
Co:  BKNG.O BKNG.OQ
assetId:  4295914598
assetName:  Booking Holdings Inc
brokerAction:  UNDEFINED
relevance:  0.0936586
sentimentClass:  1 : Postivie
sentimentPositive:  0.83362
sentimentNeutral:  0.126483
sentimentNegative:  0.0398969
priceTargetIndicator:  UNDEFINED
firstMentionSentence:  0
sentimentWordCount:  471
--------------------------------------------------------


### Windowed Count Group

The windowed count group is used to associate a count with the window of time it relates to. It is used for the noveltyCounts and volumeCounts.

#### Novelty Counts 

The novelty of the content within a news item on a particular asset is calculated by comparing it with the asset-specific text over a cache of previous news items that contain the asset.

The comparison between items is done using a linguistic fingerprint. If the news items are similar, they are termed as being “linked”. As a result, a content item can “link” only to an item of the same language.

There are five historical periods that are used in the comparison. The default periods are 12 hours, 24 hours, 3 days, 5 days and 7 days prior to the news item’s timestamp.

#### Volume Counts

The volume of news for each asset is calculated. A cache of previous news items is maintained and the number of news items that mention the asset within each of five historical periods is calculated. The cache is language-specific, e.g., a volumeCount on an English-language item measures the number of other English-language items in that historical period.

By default, the historical periods are 12 hours, 24 hours, 3 days, 5 days and 7 days prior to the news item’s timestamp and are the same used in the novelty calculations. Thus, direct comparisons between similar and total items within the historical periods can be achieved.

Example Fields:
- ```itemCount```: Number of items
- ```window```: Length of time the count covers nH (for hours) or nD (for days). Default values are “12H”, “24H”, “3D”, “5D”, and “7D”.

#### TRNA Windowed Count Processing functions

In [19]:
def windowsed_count_group(group):
    for item in group:
        print("itemCount: ", item["itemCount"])
        print("window: ", item["window"])

In [20]:
# Windowed Count - Analytics Group Fields

for analytic_score in analytic_scores_group:
    print("Novelty Counts:\n")
    windowsed_count_group(analytic_score["noveltyCounts"])
    print("--------------------------------------------------------")
    print("Volumn Counts:\n")
    windowsed_count_group(analytic_score["volumeCounts"])
    print("--------------------------------------------------------")

Novelty Counts:

itemCount:  0
window:  12H
itemCount:  0
window:  24H
itemCount:  0
window:  3D
itemCount:  0
window:  5D
itemCount:  0
window:  7D
--------------------------------------------------------
Volumn Counts:

itemCount:  2
window:  12H
itemCount:  5
window:  24H
itemCount:  7
window:  3D
itemCount:  19
window:  5D
itemCount:  34
window:  7D
--------------------------------------------------------


### Linked Id Group

The linked id group is used to associate an id with its position in a longer list of ids. It is used for the linkedIds.
This group is not populated for document-level scores, since novelty is not calculated.

Example Fields:
- ```idPosition```: Position of the linkedId in the complete list of linked Ids. 0 is the first/oldest, and the largest/most recent is the 7-day itemCount minus 1.
- ```linkedId```: id of the item at this position

In [21]:
linked_id_group = None

for analytic_score in analytic_scores_group:
    print("Linked Id Group: ")
    linked_id_group = analytic_score["linkedIds"]
    if linked_id_group:
        for linked_id in linked_id_group:
            print("idPosition: ", linked_id["idPosition"])
            print("linkedId: ", linked_id["linkedId"])

Linked Id Group: 


### News Item Group (Analytics Sub-group)

The TRNA feed contains two news item groups. This group, within the analytics group, contains values derived from the news item by the analytics system.

Example Fields:
- ```companyCount```: The number of companies explicitly listed in the news item in the subjects field
- ```exchangeAction```: One of "IMBALANCE", "HALT", "RESUME", "BLOCK TRADE", "INDICATION", "UNDEFINED".
    * Set to “UNDEFINED” for all Japanese-language scores.
- ```marketCommentary```: Indicator that the item is discussing general market conditions, such as “After the Bell” summaries.
- ```sentenceCount```: The total number of sentences in the news item.
- ```wordCount```: The total number of lexical tokens (words and punctuation) in the news item.

In [22]:
news_item_groups = _trna_messages[0]["analytics"]["newsItem"]

In [23]:
print("companyCount: ", news_item_groups["companyCount"])
print("exchangeAction: ", news_item_groups["exchangeAction"])
print("marketCommentary: ", news_item_groups["marketCommentary"])
print("sentenceCount: ", news_item_groups["sentenceCount"])
print("wordCount: ", news_item_groups["wordCount"])

companyCount:  1
exchangeAction:  UNDEFINED
marketCommentary:  False
sentenceCount:  23
wordCount:  471


## Next Steps

Once the application can retrieve each News Analytics field data from the Real-Time platform, the application needs to implement a business logic to collect and analyze those data based on interested Analytics asset. Please see the examples of how to use each asset below:
- *Sentiment*: Positive sentiment typically leads to asset price rise, negative sentiment to a decline
- *Relevance*: Filter out News Analytics records with low relevance
- *Novelty*: Filter out News Analytics records that are similar to more than 0 or 1 recent news items
- *Volume*: A sudden spike in overall news volume often leads to increased trading volume and volatility

For more detail regarding each asset usage and information, please check [News Analytics Product page](https://myaccount.lseg.com/en/product/machine-readable-news-analytics).

## References

For further details, please check out the following resources:

- [LSEG Real-Time products family page](https://developers.lseg.com/en/use-cases-catalog/real-time) on the [LSEG Developers Community](https://developers.lseg.com/) website.
- [WebSocket API page](https://developers.lseg.com/en/api-catalog/real-time-opnsrc/websocket-api).
- [Developer Webinar Recording: Introduction to Electron WebSocket API](https://www.youtube.com/watch?v=CDKWMsIQfaw).
- [News Analytics Product page](https://myaccount.lseg.com/en/product/machine-readable-news-analytics).
- [Introduction to Machine Readable News with WebSocket API](https://developers.lseg.com/en/article-catalog/article/introduction-machine-readable-news-elektron-websocket-api-refinitiv).
- [Introduction to Machine Readable News (MRN) with Enterprise Message API (EMA)](https://developers.lseg.com/en/article-catalog/article/introduction-machine-readable-news-mrn-elektron-message-api-ema).
- [MRN Data Models and Real-Time SDK Implementation Guide](https://developers.lseg.com/en/api-catalog/real-time-opnsrc/rt-sdk-java/documentation#mrn-data-models-and-elektron-implementation-guide).
- [MRN (Real-Time News) WebSocket Python example on GitHub](https://github.com/LSEG-API-Samples/Example.WebSocketAPI.Python.MRN).
- [MRN (Real-Time News) WebSocket Python Console example on GitHub](https://github.com/LSEG-API-Samples/Example.WebSocketAPI.Python.MRN.RTO)
- [MRN WebSocket JavaScript example on GitHub](https://github.com/LSEG-API-Samples/Example.WebSocketAPI.Javascript.NewsMonitor).
- [MRN WebSocket C# NewsViewer example on GitHub](https://github.com/LSEG-API-Samples/Example.WebSocketAPI.CSharp.MRNWebSocketViewer).

For any question related to this example or WebSocket API, please use the Developer Community [Q&A Forum](https://community.developers.refinitiv.com).